# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


In [3]:
file_path_list

['/home/workspace/event_data/2018-11-27-events.csv',
 '/home/workspace/event_data/2018-11-04-events.csv',
 '/home/workspace/event_data/2018-11-07-events.csv',
 '/home/workspace/event_data/2018-11-09-events.csv',
 '/home/workspace/event_data/2018-11-19-events.csv',
 '/home/workspace/event_data/2018-11-05-events.csv',
 '/home/workspace/event_data/2018-11-22-events.csv',
 '/home/workspace/event_data/2018-11-16-events.csv',
 '/home/workspace/event_data/2018-11-26-events.csv',
 '/home/workspace/event_data/2018-11-24-events.csv',
 '/home/workspace/event_data/2018-11-29-events.csv',
 '/home/workspace/event_data/2018-11-15-events.csv',
 '/home/workspace/event_data/2018-11-20-events.csv',
 '/home/workspace/event_data/2018-11-06-events.csv',
 '/home/workspace/event_data/2018-11-18-events.csv',
 '/home/workspace/event_data/2018-11-21-events.csv',
 '/home/workspace/event_data/2018-11-10-events.csv',
 '/home/workspace/event_data/2018-11-23-events.csv',
 '/home/workspace/event_data/2018-11-02-events

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [7]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

In [9]:
def run_query(query, session):
    '''
    Simple function given a query and session, that can be executed against the session
    '''
    try:
        session.execute(query)
    except Exception as e:
        print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Generate our table that allows us to get the artist and song info, given a sessionID and item in a session

### Load our data into a pandas dataframe for ease of use

In [10]:
import pandas as pd
df = pd.read_csv('event_datafile_new.csv')

## Drop table if exists

In [11]:
run_query('DROP TABLE artist_and_song_in_subsession', session)

Error from server: code=2200 [Invalid query] message="unconfigured table artist_and_song_in_subsession"


In [12]:
query = "CREATE TABLE IF NOT EXISTS artist_and_song_in_subsession "
query = query + "(session_id int, item_in_session int, artist text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))"
run_query(query, session)

### Populate the data into our table

In [13]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
for row_num, row in df.iterrows():
        query = "INSERT INTO artist_and_song_in_subsession (session_id, item_in_session, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (row['sessionId'], row['itemInSession'], row['artist'], row['song'], row['length']))

#### Verify our query returns our expected results, where we are able to get the artists, and song info, from a particular session, where we pick a item from that session
##### We can see that we use our session id, which won't be unique by iteself, along with the item_in_session as a clustering column to ensure we have unique identifiers for our data.  Which gives us the desired data

In [14]:
query = 'select artist, song_title, song_length FROM artist_and_song_in_subsession WHERE session_id = 338 and item_in_session=4'
rows = session.execute(query)
for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Drop table if exists

In [15]:
run_query('DROP TABLE userid_session_library', session)

Error from server: code=2200 [Invalid query] message="unconfigured table userid_session_library"


### Generate our table that allows us to get the song info, given a particular user during a session, where the results are sorted by the order listened to in that session

In [16]:
query = "CREATE TABLE IF NOT EXISTS userid_session_library "
query = query + "(user_id int, session_id int, item_in_session int, artist text, song_title text, user_first text, user_last text, PRIMARY KEY ((user_id, session_id), item_in_session))"
run_query(query, session)

### Populate the data into our table

In [17]:
for row_num, row in df.iterrows():
        query = "INSERT INTO userid_session_library ( user_id, session_id, item_in_session, artist, song_title, user_first, user_last)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (row['userId'], row['sessionId'], row['itemInSession'], row['artist'], row['song'],  row['firstName'], row['lastName']))

### Verify our query returns our expected results, where we are able to get the artists, and songs, for a particular user, in order they listened to during a single session
#### We can see here, that we are using our user_id partition key and session_id as our to help us get our results.  Because a user can listen to multiple songs in a session, these won't be unique.  We add in a clustering column of item_in_session to help with this.

In [18]:
query = 'select artist, song_title, user_first, user_last FROM userid_session_library WHERE user_id = 10 and session_id=182'
rows = session.execute(query)
for row in rows:
    print(row.artist, row.song_title, row.user_first, row.user_last)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Drop table if exists

In [19]:
run_query('DROP TABLE song_user_library', session)

Error from server: code=2200 [Invalid query] message="unconfigured table song_user_library"


### Generate a table that allows us to get the users who have listened to a particular song

In [20]:
query = "CREATE TABLE IF NOT EXISTS song_user_library "
query = query + "(song_name text, user_id int, user_first text, user_last text, PRIMARY KEY (song_name, user_id))"
run_query(query, session)

### Populate the data into our table

In [21]:
for row_num, row in df.iterrows():
        query = "INSERT INTO song_user_library (song_name, user_id, user_first, user_last)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (row['song'], row['userId'], row['firstName'], row['lastName']))

### Verify our query returns our expected results, where we are able to get the user info from our table, when we query against a particular song that was in our database
#### Here we are using our parition key, song_name, to help us get our results.  These wouldn't be unique by default and give us all users, so we include user_id as a clustering column.  Since we only care about knowing users who have listened to songs, and not how many times this will give us all entries.  Something for us to consider though, is our users may be listening multiple times, and we may need to eventually inlude the session, and item in session information, if there are analytics use cases there.

In [22]:
##Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "select user_first, user_last FROM song_user_library WHERE song_name='All Hands Against His Own'"
rows = session.execute(query)
for row in rows:
    print(row.user_first, row.user_last)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [23]:
run_query('DROP TABLE artist_and_song_in_subsession', session)

run_query('DROP TABLE userid_session_library', session)

run_query('DROP TABLE song_user_library', session)

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()